In [1]:
import os
# os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages mysql:mysql-connector-java:8.0.24 pyspark-shell"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-class-path /opt/workspace/mysql-connector-java-8.0.24.jar --jars /opt/workspace/mysql-connector-java-8.0.24.jar pyspark-shell'


In [2]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *

In [3]:
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()
spark

/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


22/11/24 09:58:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# User schema
schema = StructType([StructField("name", StringType(), True), StructField("age", IntegerType(), True)])

# Read file from hadoop fs
df = spark.read.option("multiline", "true").schema(schema).json("hdfs://hadoop:9000/test/user.json")

df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)



+------------+---+
|        name|age|
+------------+---+
|  Tran Van A| 99|
|Nguyen Thi B| 66|
|    Le Van C| 16|
|    Ly Thi D| 19|
+------------+---+



In [5]:
jdbcDF = spark.read \
    .format("jdbc") \
    .option('driver', 'com.mysql.jdbc.Driver') \
    .option("url", "jdbc:mysql://172.17.0.1:3306/bigdata") \
    .option("dbtable", "user") \
    .option("user", "root") \
    .option("password", "local") \
    .load()

df.write.format('jdbc').options(
          url='jdbc:mysql://172.17.0.1:3306/bigdata',
          driver='com.mysql.jdbc.Driver',
          dbtable='user',
          user='root',
          password='local').mode('append').save()

jdbcDF.show()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


+------------+---+
|        name|age|
+------------+---+
|  Tran Van A| 99|
|Nguyen Thi B| 66|
|    Le Van C| 16|
|    Ly Thi D| 19|
|  Tran Van A| 99|
|Nguyen Thi B| 66|
|    Le Van C| 16|
|    Ly Thi D| 19|
+------------+---+



In [6]:
jdbcDF.show()

+------------+---+
|        name|age|
+------------+---+
|  Tran Van A| 99|
|Nguyen Thi B| 66|
|    Le Van C| 16|
|    Ly Thi D| 19|
|  Tran Van A| 99|
|Nguyen Thi B| 66|
|    Le Van C| 16|
|    Ly Thi D| 19|
+------------+---+



mysql> SELECT * FROM user;
+--------------+------+
| name         | age  |
+--------------+------+
| Tran Van A   |   99 |
| Nguyen Thi B |   66 |
| Le Van C     |   16 |
| Ly Thi D     |   19 |
| Tran Van A   |   99 |
| Nguyen Thi B |   66 |
| Le Van C     |   16 |
| Ly Thi D     |   19 |
+--------------+------+
8 rows in set (0.01 sec)

In [7]:
spark.stop()